```
方法一:SelectKBest
SelectKBest 是 scikit-learn 中的一个自动特征选择工具，它的作用就是从一大堆特征中自动挑选出"最有用"的 K 个特征。
1. 📊 SelectKBest 分析数据 ← 训练前
2. 🎯 选出24个最佳特征
3. 🤖 用选出的特征训练模型 ← 训练阶段
对每个特征做这样的计算：
# 伪代码解释 f_regression 的计算过程
for 每个特征 in 117个特征:
    # 计算这个特征与目标变量的线性关系
    相关系数 = pearson_correlation(特征值, 目标值)
    F统计量 = (相关系数² × (样本数-2)) / (1 - 相关系数²)
    # 给这个特征打分
    特征分数[特征名] = F统计量

方法二:分析后手动选取特征值

In [1]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

# 新添加这两行用于数据分析可视化
import matplotlib.pyplot as plt
import seaborn as sns

# 在你现有的import后面添加这几行
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.linear_model import Ridge

In [2]:
# 设置当显示数据表格时，不限制显示的列数
pd.set_option('display.max_columns', None)

df_train = pd.read_csv('covid.train.csv')
# df_train.shape 数据表格有多少行、多少列
# f'train {df_train.shape}' 格式化字符串
print( f'train {df_train.shape}' )
display( df_train.head() )

train (2699, 118)


,id,AL,AK,AZ,AR,CA,CO,CT,FL,GA,ID,IL,IN,IA,KS,KY,LA,MD,MA,MI,MN,MS,MO,NE,NV,NJ,NM,NY,NC,OH,OK,OR,RI,SC,TX,UT,VA,WA,cli,ili,hh_cmnty_cli,nohh_cmnty_cli,wearing_mask,travel_outside_state,work_outside_home,shop,restaurant,spent_time,large_event,public_transit,anxious,depressed,worried_finances,tested_positive,cli.1,ili.1,hh_cmnty_cli.1,nohh_cmnty_cli.1,wearing_mask.1,travel_outside_state.1,work_outside_home.1,shop.1,restaurant.1,spent_time.1,large_event.1,public_transit.1,anxious.1,depressed.1,worried_finances.1,tested_positive.1,cli.2,ili.2,hh_cmnty_cli.2,nohh_cmnty_cli.2,wearing_mask.2,travel_outside_state.2,work_outside_home.2,shop.2,restaurant.2,spent_time.2,large_event.2,public_transit.2,anxious.2,depressed.2,worried_finances.2,tested_positive.2,cli.3,ili.3,hh_cmnty_cli.3,nohh_cmnty_cli.3,wearing_mask.3,travel_outside_state.3,work_outside_home.3,shop.3,restaurant.3,spent_time.3,large_event.3,public_transit.3,anxious.3,depressed.3,worried_finances.3,tested_positive.3,cli.4,ili.4,hh_cmnty_cli.4,nohh_cmnty_cli.4,wearing_mask.4,travel_outside_state.4,work_outside_home.4,shop.4,restaurant.4,spent_time.4,large_event.4,public_transit.4,anxious.4,depressed.4,worried_finances.4,tested_positive.4
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.658466,0.724606,12.939284,8.556230,76.521693,7.863621,33.069066,64.129378,33.438714,40.539878,17.446499,3.993190,10.122837,7.841023,37.992365,7.374846,0.653157,0.713249,12.488933,8.219380,75.179558,8.083434,32.564654,65.212903,34.285572,40.697210,17.754941,3.997443,10.161621,8.067275,37.882278,7.219989,0.677783,0.737965,12.487637,8.135510,74.189695,8.001016,32.145331,66.258549,34.931095,40.889258,18.038068,3.970457,10.043014,7.983358,37.705024,7.077938,0.666751,0.723506,12.367718,8.006131,73.211902,7.873342,31.330236,66.753902,35.586606,40.741650,17.800711,3.984791,10.148898,8.262288,37.384963,7.452243,0.685628,0.740943,12.364307,8.151578,71.820231,7.944531,31.113209,67.394551,36.674291,40.743132,17.842221,4.093712,10.440071,8.627117,37.329512,7.456154
1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.693287,0.675962,15.008467,11.091818,82.375469,11.082587,36.373964,60.471825,30.516978,43.162123,16.038761,4.519762,12.133287,10.155721,33.977549,9.850027,0.738029,0.720511,15.070049,10.990937,80.943299,10.596788,36.435828,61.653141,31.449290,43.751356,16.752518,4.385694,11.911724,9.798448,32.786802,10.050547,0.831586,0.827523,14.568504,10.578924,79.096391,10.418171,35.851804,62.835720,32.802637,43.753525,17.195922,4.671201,11.418314,10.600311,32.679030,10.388084,0.767643,0.763580,14.264862,10.212529,76.645984,10.614731,34.639961,63.652437,34.147714,44.682727,17.705140,4.938357,11.056725,10.889148,32.933142,8.707858,0.713255,0.719378,12.894363,8.919288,74.687112,10.523814,33.920257,64.398380,34.612238,44.035688,17.808103,4.924935,10.172662,9.954333,32.508881,8.010957
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.617041,0.617041,8.614719,5.238097,87.040999,15.569851,33.366362,58.819368,24.674416,36.678937,13.490946,6.639301,9.753290,7.691015,38.685771,3.897851,0.663597,0.635459,8.472831,5.154184,86.743467,15.915242,33.012910,60.157822,25.322821,36.456240,13.385945,6.700146,9.949982,7.833514,38.036480,3.083111,0.534394,0.497435,8.493353,5.125553,86.647289,15.963496,32.471093,60.910411,25.158967,35.968207,13.656070,6.748557,10.664876,7.975821,37.826924,3.132834,0.536224,0.498305,8.227657,5.298912,86.285798,15.788676,31.931642,61.233289,25.156025,36.032687,13.863302,6.879646,10.731187,7.847829,37.128714,3.444182,0.479111,0.432445,8.067909,5.333533,86.312419,16.477784,31.604604,62.101064,26.521875,36.746453,13.903667,7.313833,10.388712,7.956139,36.745588,2.906977
3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.193768,1.272051,12.007656,7.219741,43.153686,20.972589,35.458649,69.833774,37.021123,48.022187,28.436903,4.083615,9.417865,9.385772,37.621189,10.297712,1.356577,1.432775,1

In [3]:
df = pd.read_csv('covid.train.csv',index_col=0)
test_df = pd.read_csv('covid.test.csv',index_col=0)

In [4]:
# ========== 数据分析代码==========
# 读取数据
df = pd.read_csv('covid.train.csv', index_col=0)
test_df = pd.read_csv('covid.test.csv', index_col=0)

# 定义目标变量
target_col = df.columns[-1]
print(f"目标变量: {target_col}")

print("="*50)
print("📊 数据基本信息")
print("="*50)
print(f"训练数据形状: {df.shape}")
print(f"测试数据形状: {test_df.shape}")
print(f"总特征数量: {df.shape[1]-1}")

# 特征类型识别
print("\n" + "="*50)
print("🔍 特征类型识别")
print("="*50)

potential_state_features = []
numerical_features = []

for col in df.columns[:-1]:  # 不包括目标变量
    unique_vals = df[col].nunique()
    if unique_vals <= 2:  # 二进制特征，可能是州信息
        potential_state_features.append(col)
    else:
        numerical_features.append(col)

print(f"可能的州信息特征数量: {len(potential_state_features)}")
print(f"数值型特征数量: {len(numerical_features)}")

# 相关性分析
print("\n" + "="*50)
print("📈 相关性分析")
print("="*50)

# 计算所有特征与目标变量的相关性
all_correlations = df.corr()[target_col].sort_values(ascending=False)

# 分别分析州特征和数值特征的相关性
state_correlations = all_correlations[potential_state_features]
numerical_correlations = all_correlations[numerical_features]

print("州信息特征与目标变量的相关性:")
print(f"最高相关性: {state_correlations.max():.4f}")
print(f"最低相关性: {state_correlations.min():.4f}")
print(f"平均相关性: {state_correlations.mean():.4f}")

print("\n数值特征与目标变量的相关性:")
print("前5个高相关性特征:")
top5_numerical = numerical_correlations.head(5)
for feat, corr in top5_numerical.items():
    print(f"  {feat}: {corr:.4f}")

# 筛选高相关性的数值特征
high_corr_features = numerical_correlations[abs(numerical_correlations) > 0.3]
print(f"\n高相关性特征(|相关系数| > 0.3): {len(high_corr_features)}个")


目标变量: tested_positive.4
📊 数据基本信息
训练数据形状: (2699, 117)
测试数据形状: (1078, 116)
总特征数量: 116

🔍 特征类型识别
可能的州信息特征数量: 37
数值型特征数量: 79

📈 相关性分析
州信息特征与目标变量的相关性:
最高相关性: 0.3243
最低相关性: -0.1762
平均相关性: -0.0000

数值特征与目标变量的相关性:
前5个高相关性特征:
  cli: 0.8528
  ili: 0.8529
  hh_cmnty_cli: 0.8808
  nohh_cmnty_cli: 0.8738
  wearing_mask: -0.1359

高相关性特征(|相关系数| > 0.3): 53个


In [5]:
# ========== 简化版：只分析前4个高相关性特征 ==========
print("\n" + "="*50)
print("🔗 前4个重要特征的相关性分析")
print("="*50)
# 提取前4个高相关性特征
top4_features = ['cli', 'ili', 'hh_cmnty_cli', 'nohh_cmnty_cli']
print("分析的特征:", top4_features)
# 计算这4个特征之间的相关性矩阵
top4_df = df[top4_features]
top4_corr_matrix = top4_df.corr()
print(f"\n📊 特征间相关性矩阵:")
print(top4_corr_matrix.round(4))
# 找出高度相关的特征对
print(f"\n🔗 高度相关的特征对 (|相关系数| > 0.8):")
high_corr_count = 0
redundant_features = set()
for i in range(len(top4_features)):
    for j in range(i+1, len(top4_features)):
        corr_value = top4_corr_matrix.iloc[i, j]
        if abs(corr_value) > 0.8:
            feat1, feat2 = top4_features[i], top4_features[j]
            print(f"  {feat1} <-> {feat2}: {corr_value:.4f}")
            high_corr_count += 1
            
            # 决定保留哪个特征（保留与目标变量相关性更高的）
            corr1 = abs(numerical_correlations[feat1])
            corr2 = abs(numerical_correlations[feat2])
            if corr1 < corr2:
                redundant_features.add(feat1)
            else:
                redundant_features.add(feat2)
if high_corr_count == 0:
    print("  没有发现高度相关的特征对")


🔗 前4个重要特征的相关性分析
分析的特征: ['cli', 'ili', 'hh_cmnty_cli', 'nohh_cmnty_cli']

📊 特征间相关性矩阵:
                   cli     ili  hh_cmnty_cli  nohh_cmnty_cli
cli             1.0000  0.9969        0.9283          0.9221
ili             0.9969  1.0000        0.9290          0.9223
hh_cmnty_cli    0.9283  0.9290        1.0000          0.9967
nohh_cmnty_cli  0.9221  0.9223        0.9967          1.0000

🔗 高度相关的特征对 (|相关系数| > 0.8):
  cli <-> ili: 0.9969
  cli <-> hh_cmnty_cli: 0.9283
  cli <-> nohh_cmnty_cli: 0.9221
  ili <-> hh_cmnty_cli: 0.9290
  ili <-> nohh_cmnty_cli: 0.9223
  hh_cmnty_cli <-> nohh_cmnty_cli: 0.9967
